In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.model_selection import GridSearchCV

In [4]:
df = pd.read_csv('sales_data.csv')

In [5]:
df.columns

Index(['date', 'channel', 'country', 'shop_name', 'sku', 'product_name',
       'brand', 'url', 'sap', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6',
       'sales', 'units_sold', 'product_views', 'click_rate', 'buyer', 'order'],
      dtype='object')

In [6]:
df = df.drop(['cat1', 'cat2', 'cat3', 'cat4', 
              'cat5', 'cat6', 'buyer', 'order',
              'country', 'url' ], axis=1)

In [7]:
df = df.drop(['buyer', 'order'], axis=1)

In [8]:
df = df.drop(['country', 'url'], axis=1)

In [9]:
df.head()

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,Lactum 3+ Plain Powdered Milk Drink 2kg,Lactum,2048174.0,0.0,0,25,0.00
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,Lactum 3+ Plain Powdered Milk Drink 1.6Kg with...,Lactum,0.0,0.0,10,116,8.62
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,Lactum 3+ Plain Powdered Milk Drink 1.2Kg with...,Lactum,0.0,0.0,1,27,3.70
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,Lactum 3+ Plain Powdered Milk Drink 150g,Lactum,2020877.0,0.0,0,62,0.00
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 Packs Of Lactum 3+ Plain Powdered Milk Drink...,Lactum,0.0,0.0,17,298,5.70


## Feature Engineering - Dates

In [10]:
df.date = pd.to_datetime(df.date)

In [11]:
df.head()

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,Lactum 3+ Plain Powdered Milk Drink 2kg,Lactum,2048174.0,0.0,0,25,0.00
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,Lactum 3+ Plain Powdered Milk Drink 1.6Kg with...,Lactum,0.0,0.0,10,116,8.62
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,Lactum 3+ Plain Powdered Milk Drink 1.2Kg with...,Lactum,0.0,0.0,1,27,3.70
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,Lactum 3+ Plain Powdered Milk Drink 150g,Lactum,2020877.0,0.0,0,62,0.00
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 Packs Of Lactum 3+ Plain Powdered Milk Drink...,Lactum,0.0,0.0,17,298,5.70


In [12]:
months = df.date.dt.month
days = df.date.dt.day
day_name = df.date.dt.day_name()
is_weekend = day_name.apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

In [13]:
df1 = df.copy()

In [14]:
#dates
df_days = pd.get_dummies(days, prefix='day')
df_months = pd.get_dummies(months, prefix='month')
df_dayname = pd.get_dummies(day_name)
df_weekend = pd.get_dummies(is_weekend, prefix='weekend')
df_datefeatures = pd.concat([df_months, df_days, df_dayname, df_weekend], axis=1)

In [91]:
def agegroup(milk):
    if '0-6' in milk:
        age = '0-6'
    elif '6-12' in milk:
        age = '6-12'
    elif '0-12' in milk:
        age = '0-12'
    elif '1-3' in milk:
        age = '1-3'
    elif '3+' in milk:
        age = '3+'
    elif '6+' in milk:
        age = '6+'
    else:
        age = 'enfamama'
    return age
def flavor(item):
    if 'chocolate' in item:
        flav = 'chocolate'
    elif 'choco' in item:
        flav = 'chocolate'
    elif 'vanilla' in item:
        flav = 'vanilla'
    elif 'milk' in item:
        flav = 'milk'
    elif 'formula' in item:
        flav = 'formula'
    elif 'plain' in item:
        flav = 'plain'
    elif 'a+ four' in item:
        flav = 'plain'
    elif 'premium adult nutritional beverage' in item:
        flav = 'milk'
    elif 'premium adult nutrional beverage' in item:
        flav = 'milk'
    elif 'premium adult nutrional beverage' in item:
        flav = 'milk'
    elif 'premium adult nutrition beverage' in item:
        flav = 'milk'
    elif 'gentlease' in item:
        flav = 'milk'
    elif 'a+ three' in item:
        flav = 'milk'
    else:
        flav = 'unknown'
    return flav


def bundle(bund):
    if '[' in bund:
        bund1 =  1
    elif 'buy' in bund:
        if 'buy 1 ' in bund:
            bund1 = 0
        else:
            bund1 = 1
    elif 'x 3' in bund:
        bund1 = 1
    else:
        bund1 = 0
    return bund1

In [154]:
df_copy = df.copy()

In [155]:
df_copy.head()

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,Lactum 3+ Plain Powdered Milk Drink 2kg,Lactum,2048174.0,0.0,0,25,0.00
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,Lactum 3+ Plain Powdered Milk Drink 1.6Kg with...,Lactum,0.0,0.0,10,116,8.62
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,Lactum 3+ Plain Powdered Milk Drink 1.2Kg with...,Lactum,0.0,0.0,1,27,3.70
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,Lactum 3+ Plain Powdered Milk Drink 150g,Lactum,2020877.0,0.0,0,62,0.00
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 Packs Of Lactum 3+ Plain Powdered Milk Drink...,Lactum,0.0,0.0,17,298,5.70


In [156]:
df_copy['age_group']=df_copy['product_name'].apply(lambda x: agegroup(x))
df_copy['product_name'] = df_copy['product_name'].apply(lambda x: x.lower())
df_copy['flavor'] = df_copy['product_name'].apply(lambda x: flavor(x))
df_copy['bundle'] = df_copy['product_name'].apply(lambda x: bundle(x))

In [160]:
df_copy.head()

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate,age_group,flavor,bundle
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,lactum 3+ plain powdered milk drink 2kg,Lactum,2048174.0,0.0,0,25,0.00,3+,milk,0
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,lactum 3+ plain powdered milk drink 1.6kg with...,Lactum,0.0,0.0,10,116,8.62,3+,milk,0
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,lactum 3+ plain powdered milk drink 1.2kg with...,Lactum,0.0,0.0,1,27,3.70,3+,milk,0
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,lactum 3+ plain powdered milk drink 150g,Lactum,2020877.0,0.0,0,62,0.00,3+,milk,0
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 packs of lactum 3+ plain powdered milk drink...,Lactum,0.0,0.0,17,298,5.70,3+,milk,0


In [161]:
df1 = df_copy.copy()

In [164]:
df1.shape

(93007, 14)

In [165]:
months = df1.date.dt.month
days = df1.date.dt.day
day_name = df1.date.dt.day_name()
is_weekend = day_name.apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

In [166]:
#channel
channel = pd.get_dummies(df1['channel'])

In [16]:
#categorical binning
df1.shop_name.unique()

array(['Lazada Retail Lactum', 'Lazada Retail Enfagrow',
       'Lazada Retail Sustagen', 'Lactum Official Store',
       'Enfagrow A+ Four Official Store', 'Sustagen Official Store',
       'LazMart Philippines', 'lactumofficialstoreph',
       'Enfagrow Official Store_PH', 'sustagenofficialstoreph',
       ' Lactum Official Store', ' Enfagrow Official Store'], dtype=object)

In [167]:
shop_conditions = [
    df1['shop_name'].str.contains('Lactum', 'lactum'),
    df1['shop_name'].str.contains('Enfagrow'),
    df1['shop_name'].str.contains('Sustagen', 'sustagen')
]

shops = ['Lactum Store', 'Enfagrow Store', 'Sustagen Store']

shop_bins = np.select(shop_conditions, shops, default='Other')


In [168]:
df1 = df1.drop(['sku', 'product_name', 'brand'], axis=1)

In [169]:
df1 = df1.drop('product_name', axis=1)

In [170]:
df1 = df1.drop('brand', axis=1)

In [182]:
df1

,shop_name,channel,sales,units_sold,product_views,click_rate,age_group_0-12,age_group_0-6,age_group_1-3,age_group_3+,...,age_group_6-12,age_group_enfamama,flavor_chocolate,flavor_formula,flavor_milk,flavor_plain,flavor_unknown,flavor_vanilla,bundle_0,bundle_1
0,Lactum Official Store,Shopee,86.0,6,0,0.00,0,0,2,2,...,0,0,0,0,4,0,0,0,4,0
1,Lazada Retail Enfagrow,Lazada,155.0,11,564,46.07,1,4,4,14,...,6,2,1,5,24,0,0,1,31,0
2,Lazada Retail Lactum,Lazada,30.0,36,985,96.30,0,2,3,16,...,3,0,5,2,24,0,0,0,31,0
3,Lazada Retail Sustagen,Lazada,0.0,0,50,0.00,0,0,0,0,...,0,6,3,0,3,0,0,0,6,0
4,Enfagrow A+ Four Official Store,Shopee,30.0,1,0,0.00,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,LazMart Philippines,Lazada,0.0,0,0,0.00,0,4,5,6,...,3,26,0,0,6,4,41,0,51,0
4572,Lazada Retail Enfagrow,Lazada,0.0,0,0,123.73,3,5,9,8,...,4,4,1,0,14,7,10,1,32,1
4573,Lazada Retail Lactum,Lazada,0.0,0,0,201.01,0,3,8,12,...,3,0,5,0,8,1,21,0,33,2
4574,Lazada Retail Sustagen,Lazada,0.0,0,0,14.34,0,0,0,0,...,0,8,5,0,3,0,0,0,8,0


In [183]:
shop_conditions = [
    df1['shop_name'].str.contains('Lactum', 'lactum'),
    df1['shop_name'].str.contains('Enfagrow'),
    df1['shop_name'].str.contains('Sustagen', 'sustagen')
]

shops = ['Lactum Store', 'Enfagrow Store', 'Sustagen Store']

shop_bins = np.select(shop_conditions, shops, default='Other')

In [172]:
df1 = pd.get_dummies(data=df1, columns=['age_group', 'flavor',  'bundle'])

In [173]:
df1 = df1.drop('sap', axis=1)

In [174]:
df1.columns

Index(['date', 'channel', 'shop_name', 'sales', 'units_sold', 'product_views',
       'click_rate', 'age_group_0-12', 'age_group_0-6', 'age_group_1-3',
       'age_group_3+', 'age_group_6+', 'age_group_6-12', 'age_group_enfamama',
       'flavor_chocolate', 'flavor_formula', 'flavor_milk', 'flavor_plain',
       'flavor_unknown', 'flavor_vanilla', 'bundle_0', 'bundle_1'],
      dtype='object')

In [175]:
df1 = df1.groupby(['date', 
             'shop_name',
             'channel'])[df1.columns[3:]].sum().reset_index()

In [124]:
df1.groupby(['date', 
             'shop_name',
             'channel']).agg(sales=('sales','sum'),
                             sap=('sap','sum'),
                             units_sold=('units_sold', 'sum'),
                             product_views=('product_views', 'sum')).reset_index()

,date,shop_name,channel,sales,sap,units_sold,product_views
0,2019-01-01,Lactum Official Store,Shopee,86.0,8083473.0,6,0
1,2019-01-01,Lazada Retail Enfagrow,Lazada,155.0,36683478.0,11,564
2,2019-01-01,Lazada Retail Lactum,Lazada,30.0,38670820.0,36,985
3,2019-01-01,Lazada Retail Sustagen,Lazada,0.0,2496645.0,0,50
4,2019-01-02,Enfagrow A+ Four Official Store,Shopee,30.0,2038651.0,1,0
...,...,...,...,...,...,...,...
4571,2020-12-31,LazMart Philippines,Lazada,0.0,105119296.0,0,0
4572,2020-12-31,Lazada Retail Enfagrow,Lazada,0.0,57290357.0,0,0
4573,2020-12-31,Lazada Retail Lactum,Lazada,0.0,57299575.0,0,0
4574,2020-12-31,Lazada Retail Sustagen,Lazada,0.0,7048817.0,0,0


In [153]:
df1

,shop_name,channel,sales,units_sold,product_views,click_rate,age_group_0-12,age_group_0-6,age_group_1-3,age_group_3+,...,age_group_6-12,age_group_enfamama,flavor_chocolate,flavor_formula,flavor_milk,flavor_plain,flavor_unknown,flavor_vanilla,bundle_0,bundle_1
0,Lactum Official Store,Shopee,86.0,6,0,0.00,0,0,2,2,...,0,0,0,0,4,0,0,0,4,0
1,Lazada Retail Enfagrow,Lazada,155.0,11,564,46.07,1,4,4,14,...,6,2,1,5,24,0,0,1,31,0
2,Lazada Retail Lactum,Lazada,30.0,36,985,96.30,0,2,3,16,...,3,0,5,2,24,0,0,0,31,0
3,Lazada Retail Sustagen,Lazada,0.0,0,50,0.00,0,0,0,0,...,0,6,3,0,3,0,0,0,6,0
4,Enfagrow A+ Four Official Store,Shopee,30.0,1,0,0.00,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,LazMart Philippines,Lazada,0.0,0,0,0.00,0,4,5,6,...,3,26,0,0,6,4,41,0,51,0
4572,Lazada Retail Enfagrow,Lazada,0.0,0,0,123.73,3,5,9,8,...,4,4,1,0,14,7,10,1,32,1
4573,Lazada Retail Lactum,Lazada,0.0,0,0,201.01,0,3,8,12,...,3,0,5,0,8,1,21,0,33,2
4574,Lazada Retail Sustagen,Lazada,0.0,0,0,14.34,0,0,0,0,...,0,8,5,0,3,0,0,0,8,0


In [176]:
df1

,date,shop_name,channel,sales,units_sold,product_views,click_rate,age_group_0-12,age_group_0-6,age_group_1-3,...,age_group_6-12,age_group_enfamama,flavor_chocolate,flavor_formula,flavor_milk,flavor_plain,flavor_unknown,flavor_vanilla,bundle_0,bundle_1
0,2019-01-01,Lactum Official Store,Shopee,86.0,6,0,0.00,0,0,2,...,0,0,0,0,4,0,0,0,4,0
1,2019-01-01,Lazada Retail Enfagrow,Lazada,155.0,11,564,46.07,1,4,4,...,6,2,1,5,24,0,0,1,31,0
2,2019-01-01,Lazada Retail Lactum,Lazada,30.0,36,985,96.30,0,2,3,...,3,0,5,2,24,0,0,0,31,0
3,2019-01-01,Lazada Retail Sustagen,Lazada,0.0,0,50,0.00,0,0,0,...,0,6,3,0,3,0,0,0,6,0
4,2019-01-02,Enfagrow A+ Four Official Store,Shopee,30.0,1,0,0.00,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,2020-12-31,LazMart Philippines,Lazada,0.0,0,0,0.00,0,4,5,...,3,26,0,0,6,4,41,0,51,0
4572,2020-12-31,Lazada Retail Enfagrow,Lazada,0.0,0,0,123.73,3,5,9,...,4,4,1,0,14,7,10,1,32,1
4573,2020-12-31,Lazada Retail Lactum,Lazada,0.0,0,0,201.01,0,3,8,...,3,0,5,0,8,1,21,0,33,2
4574,2020-12-31,Lazada Retail Sustagen,Lazada,0.0,0,0,14.34,0,0,0,...,0,8,5,0,3,0,0,0,8,0


In [177]:
months = df1.date.dt.month
days = df1.date.dt.day
day_name = df1.date.dt.day_name()
is_weekend = day_name.apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

In [178]:
#dates
df_days = pd.get_dummies(days, prefix='day')
df_months = pd.get_dummies(months, prefix='month')
df_dayname = pd.get_dummies(day_name)
df_weekend = pd.get_dummies(is_weekend, prefix='weekend')
df_datefeatures = pd.concat([df_months, df_days, df_dayname, df_weekend], axis=1)

In [179]:
df_datefeatures

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,day_31,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekend_0,weekend_1
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
4572,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
4573,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
4574,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0


,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,day_31,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,weekend_0,weekend_1
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93002,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
93003,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
93004,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
93005,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0


In [180]:
df1 = df1.drop('date', axis=1)

In [181]:
df1.head()

,shop_name,channel,sales,units_sold,product_views,click_rate,age_group_0-12,age_group_0-6,age_group_1-3,age_group_3+,...,age_group_6-12,age_group_enfamama,flavor_chocolate,flavor_formula,flavor_milk,flavor_plain,flavor_unknown,flavor_vanilla,bundle_0,bundle_1
0,Lactum Official Store,Shopee,86.0,6,0,0.00,0,0,2,2,...,0,0,0,0,4,0,0,0,4,0
1,Lazada Retail Enfagrow,Lazada,155.0,11,564,46.07,1,4,4,14,...,6,2,1,5,24,0,0,1,31,0
2,Lazada Retail Lactum,Lazada,30.0,36,985,96.30,0,2,3,16,...,3,0,5,2,24,0,0,0,31,0
3,Lazada Retail Sustagen,Lazada,0.0,0,50,0.00,0,0,0,0,...,0,6,3,0,3,0,0,0,6,0
4,Enfagrow A+ Four Official Store,Shopee,30.0,1,0,0.00,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0


In [186]:
df2 = df1.copy()

In [189]:
df2 = pd.get_dummies(data=df2, columns=['shop_name', 'channel'])

In [191]:
df2 = pd.concat([df_datefeatures, df2], axis=1)

In [144]:
df2 = pd.concat([df2, shops], axis=1)

In [192]:
df2.sales

0         86.0
1        155.0
2         30.0
3          0.0
4         30.0
         ...  
4571       0.0
4572       0.0
4573       0.0
4574       0.0
4575    1350.0
Name: sales, Length: 4576, dtype: float64

In [194]:
df2.columns

Index(['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8',
       'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15',
       'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21', 'day_22',
       'day_23', 'day_24', 'day_25', 'day_26', 'day_27', 'day_28', 'day_29',
       'day_30', 'day_31', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'weekend_0', 'weekend_1', 'sales',
       'units_sold', 'product_views', 'click_rate', 'age_group_0-12',
       'age_group_0-6', 'age_group_1-3', 'age_group_3+', 'age_group_6+',
       'age_group_6-12', 'age_group_enfamama', 'flavor_chocolate',
       'flavor_formula', 'flavor_milk', 'flavor_plain', 'flavor_unknown',
       'flavor_vanilla', 'bundle_0', 'bundle_1', 'shop_name_Enfagrow Store',
       'shop_name_Lactum Store', '

In [195]:
df2 = df2.drop(['product_views', 'units_sold', 'click_rate'], axis=1)

In [215]:
df.date[1]

Timestamp('2019-01-01 00:00:00')

In [223]:
#raw_data.loc[raw_data['date_field'] >= '2019-06-20'

df2.columns

Index(['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8',
       'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15',
       'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21', 'day_22',
       'day_23', 'day_24', 'day_25', 'day_26', 'day_27', 'day_28', 'day_29',
       'day_30', 'day_31', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'weekend_0', 'weekend_1', 'sales',
       'age_group_0-12', 'age_group_0-6', 'age_group_1-3', 'age_group_3+',
       'age_group_6+', 'age_group_6-12', 'age_group_enfamama',
       'flavor_chocolate', 'flavor_formula', 'flavor_milk', 'flavor_plain',
       'flavor_unknown', 'flavor_vanilla', 'bundle_0', 'bundle_1',
       'shop_name_Enfagrow Store', 'shop_name_Lactum Store', 'shop_name_Other',
       'shop_name_Sustagen Store',

In [196]:
df_target = df2['sales']
df_features = df2.drop('sales', axis=1)

In [224]:
# df_sp_concur = df_sp.groupby('date')['campaign'].count().reset_index()
# df_sp_concur = df_sp_concur.rename(columns={"campaign": "concurrent_campaign"})
# df_sp_concur

df

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,Lactum 3+ Plain Powdered Milk Drink 2kg,Lactum,2048174.0,0.0,0,25,0.00
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,Lactum 3+ Plain Powdered Milk Drink 1.6Kg with...,Lactum,0.0,0.0,10,116,8.62
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,Lactum 3+ Plain Powdered Milk Drink 1.2Kg with...,Lactum,0.0,0.0,1,27,3.70
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,Lactum 3+ Plain Powdered Milk Drink 150g,Lactum,2020877.0,0.0,0,62,0.00
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 Packs Of Lactum 3+ Plain Powdered Milk Drink...,Lactum,0.0,0.0,17,298,5.70
...,...,...,...,...,...,...,...,...,...,...,...
93002,2020-12-31,Lazada,Lazada Retail Sustagen,298062323_PH-508740875,Sustagen Premium Chocolate Adult Nutritional B...,Sustagen Premium,1248323.0,0.0,0,0,0.00
93003,2020-12-31,Lazada,Lazada Retail Sustagen,298062315_PH-508738075,Sustagen Premium Adult Nutritional Beverage 350g,Sustagen Premium,1248320.0,0.0,0,0,0.00
93004,2020-12-31,Lazada,Lazada Retail Sustagen,298058375_PH-508722170,Sustagen Premium Adult Nutritional Beverage 900g,Sustagen Premium,1248321.0,0.0,0,0,0.00
93005,2020-12-31,Lazada,Lazada Retail Sustagen,1286610071_PH-4668890342,Sustagen Premium 9.6kg Adult Nutritional Bever...,Sustagen Premium,0.0,0.0,0,0,0.00


In [200]:
df_features.columns

Index(['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8',
       'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15',
       'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21', 'day_22',
       'day_23', 'day_24', 'day_25', 'day_26', 'day_27', 'day_28', 'day_29',
       'day_30', 'day_31', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'weekend_0', 'weekend_1',
       'age_group_0-12', 'age_group_0-6', 'age_group_1-3', 'age_group_3+',
       'age_group_6+', 'age_group_6-12', 'age_group_enfamama',
       'flavor_chocolate', 'flavor_formula', 'flavor_milk', 'flavor_plain',
       'flavor_unknown', 'flavor_vanilla', 'bundle_0', 'bundle_1',
       'shop_name_Enfagrow Store', 'shop_name_Lactum Store', 'shop_name_Other',
       'shop_name_Sustagen Store', 'channel

In [197]:
X_train_0_1, X_test_0_1, y_train_0_1, y_test_0_1 = train_test_split(
    df_features, df_target, test_size=0.25)

In [201]:
#For purpose of Demonstration let's just use the default setting of the following models
LR = LinearRegression()
LR.fit(X_train_0_1, y_train_0_1)
Lasso1 = Lasso()
Lasso1.fit(X_train_0_1, y_train_0_1)
Ridge1 = Ridge()
Ridge1.fit(X_train_0_1, y_train_0_1)
#kNN = KNeighborsRegressor(n_neighbors=8) 
kNN = KNeighborsRegressor() 
kNN.fit(X_train_0_1,y_train_0_1)
#DT = DecisionTreeRegressor(max_depth=5) 
DT = DecisionTreeRegressor() 
DT.fit(X_train_0_1,y_train_0_1)
#RF = RandomForestRegressor(n_estimators = 100) 
RF = RandomForestRegressor() 
RF.fit(X_train_0_1,y_train_0_1)
#GBM = GradientBoostingRegressor(max_depth =5, learning_rate=.1) 
GBM = GradientBoostingRegressor() 
GBM.fit(X_train_0_1,y_train_0_1)

# y_test_LR = LR.predict(X_test)
# y_test_Lasso = Lasso.predict(X_test)
# y_test_Ridge = Ridge1.predict(X_test)
# y_test_kNN = kNN.predict(X_test)
# y_test_DT = DT.predict(X_test)
# y_test_RF = RF.predict(X_test)
# y_test_GBM = GBM.predict(X_test)


cols = ['Regression Method','Train Accuracy', 'Test Accuracy']
df_0_1 = pd.DataFrame(columns=cols)

df_0_1.loc[0] = ['Linear Regression', LR.score(X_train_0_1, y_train_0_1), LR.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[1] = ['Linear Regression + Lasso', Lasso1.score(X_train_0_1, y_train_0_1), Lasso1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[2] = ['Linear Regression + Ridge', Ridge1.score(X_train_0_1, y_train_0_1), Ridge1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[3] = ['kNN', kNN.score(X_train_0_1, y_train_0_1), kNN.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[4] = ['Decision Tree',DT.score(X_train_0_1, y_train_0_1), DT.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[5] = ['Random Forest',RF.score(X_train_0_1, y_train_0_1), RF.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[6] = ['Gradient Boosting Method',GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1)]

df_0_1

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19664284020.875183, tolerance: 32899263.9423297
  model = cd_fast.enet_coordinate_descent(


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.147425,0.158384
1,Linear Regression + Lasso,0.147409,0.158811
2,Linear Regression + Ridge,0.147421,0.158459
3,kNN,0.390272,0.081269
4,Decision Tree,1.000000,-0.285505
5,Random Forest,0.899938,0.263327
6,Gradient Boosting Method,0.811651,0.239705


---

In [93]:
df1.head()

,channel,shop_name,sap,sales,units_sold,product_views,click_rate
0,Lazada,Lactum Store,2048174.0,0.0,0,25,0.00
1,Lazada,Lactum Store,0.0,0.0,10,116,8.62
2,Lazada,Lactum Store,0.0,0.0,1,27,3.70
3,Lazada,Lactum Store,2020877.0,0.0,0,62,0.00
4,Lazada,Lactum Store,0.0,0.0,17,298,5.70


In [47]:
df2

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,weekend_1,Lazada,Shopee,shop_name,channel,sales,click_rate,sap,units_sold,product_views
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,Enfagrow Store,Lazada,155.00,1.486129,36683478.0,11,564
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,Lactum Store,Lazada,30.00,3.106452,38670820.0,36,985
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,Lactum Store,Shopee,86.00,0.000000,8083473.0,6,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,Sustagen Store,Lazada,0.00,0.000000,2496645.0,0,50
4,1,0,0,0,0,0,0,0,0,0,...,0,1,0,Enfagrow Store,Lazada,644.00,2.322903,36683478.0,35,962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0,0,0,0,0,0,0,0,0,0,...,0,1,0,Lactum Store,Lazada,0.00,5.743143,57299575.0,0,0
4425,0,0,0,0,0,0,0,0,0,0,...,0,0,1,Lactum Store,Shopee,2302.67,9.925455,12211125.0,0,960
4426,0,0,0,0,0,0,0,0,0,0,...,0,1,0,Other,Lazada,0.00,0.000000,105119296.0,0,0
4427,0,0,0,0,0,0,0,0,0,0,...,0,0,1,Other,Shopee,1350.00,25.935556,0.0,0,394


In [229]:
df

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,Lactum 3+ Plain Powdered Milk Drink 2kg,Lactum,2048174.0,0.0,0,25,0.00
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,Lactum 3+ Plain Powdered Milk Drink 1.6Kg with...,Lactum,0.0,0.0,10,116,8.62
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,Lactum 3+ Plain Powdered Milk Drink 1.2Kg with...,Lactum,0.0,0.0,1,27,3.70
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,Lactum 3+ Plain Powdered Milk Drink 150g,Lactum,2020877.0,0.0,0,62,0.00
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 Packs Of Lactum 3+ Plain Powdered Milk Drink...,Lactum,0.0,0.0,17,298,5.70
...,...,...,...,...,...,...,...,...,...,...,...
93002,2020-12-31,Lazada,Lazada Retail Sustagen,298062323_PH-508740875,Sustagen Premium Chocolate Adult Nutritional B...,Sustagen Premium,1248323.0,0.0,0,0,0.00
93003,2020-12-31,Lazada,Lazada Retail Sustagen,298062315_PH-508738075,Sustagen Premium Adult Nutritional Beverage 350g,Sustagen Premium,1248320.0,0.0,0,0,0.00
93004,2020-12-31,Lazada,Lazada Retail Sustagen,298058375_PH-508722170,Sustagen Premium Adult Nutritional Beverage 900g,Sustagen Premium,1248321.0,0.0,0,0,0.00
93005,2020-12-31,Lazada,Lazada Retail Sustagen,1286610071_PH-4668890342,Sustagen Premium 9.6kg Adult Nutritional Bever...,Sustagen Premium,0.0,0.0,0,0,0.00


In [55]:
est = GradientBoostingRegressor()
param_grids = {'learning_rate': [.2, 0.1, 0.05, 0.02, 0.01],
              'max_depth': [3, 4, 6, 10, 14],
              'min_samples_leaf': [2, 3, 4],
              'max_features': [.5,.3, .2] 
}   
gs_cv = GridSearchCV(est, param_grids).fit(X_train_0_1, y_train_0_1)
print(gs_cv.best_params_)

KeyboardInterrupt: 

In [68]:
clf.cv_results_

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,Lazada,Shopee,sales,click_rate,channel_Lazada,channel_Shopee,shop_name_Enfagrow Store,shop_name_Lactum Store,shop_name_Other,shop_name_Sustagen Store
0,1,0,0,0,0,0,0,0,0,0,...,1,0,155.00,1.486129,1,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,30.00,3.106452,1,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,86.00,0.000000,0,1,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0.00,0.000000,1,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,1,0,644.00,2.322903,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0,0,0,0,0,0,0,0,0,0,...,1,0,0.00,5.743143,1,0,0,1,0,0
4425,0,0,0,0,0,0,0,0,0,0,...,0,1,2302.67,9.925455,0,1,0,1,0,0
4426,0,0,0,0,0,0,0,0,0,0,...,1,0,0.00,0.000000,1,0,0,0,1,0
4427,0,0,0,0,0,0,0,0,0,0,...,0,1,1350.00,25.935556,0,1,0,0,1,0


In [59]:
df2

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,Shopee,sales,click_rate,product_views,channel_Lazada,channel_Shopee,shop_name_Enfagrow Store,shop_name_Lactum Store,shop_name_Other,shop_name_Sustagen Store
0,1,0,0,0,0,0,0,0,0,0,...,0,155.00,1.486129,564,1,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,30.00,3.106452,985,1,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,1,86.00,0.000000,0,0,1,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0.00,0.000000,50,1,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,644.00,2.322903,962,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0,0,0,0,0,0,0,0,0,0,...,0,0.00,5.743143,0,1,0,0,1,0,0
4425,0,0,0,0,0,0,0,0,0,0,...,1,2302.67,9.925455,960,0,1,0,1,0,0
4426,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.000000,0,1,0,0,0,1,0
4427,0,0,0,0,0,0,0,0,0,0,...,1,1350.00,25.935556,394,0,1,0,0,1,0


In [139]:
cols = ['Regression Method','Train Accuracy', 'Test Accuracy']
df_0_1 = pd.DataFrame(columns=cols)

df_0_1.loc[3] = ['kNN', kNN.score(X_train_0_1, y_train_0_1), kNN.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[4] = ['Decision Tree',DT.score(X_train_0_1, y_train_0_1), DT.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[5] = ['Random Forest',RF.score(X_train_0_1, y_train_0_1), RF.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[6] = ['Gradient Boosting Method',GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1)]

df_0_1

,Regression Method,Train Accuracy,Test Accuracy
3,kNN,0.630728,0.488919
4,Decision Tree,0.992981,0.151916
5,Random Forest,0.916078,0.510718
6,Gradient Boosting Method,0.923403,0.552112


In [26]:
df1.shop_name.unique()

array(['Lactum Store', 'Enfagrow Store', 'Sustagen Store', 'Other'],
      dtype=object)